# Climate change's impact on marine food webs
### Tyler Kloster, Hana Burroughs
  
Expanding on a paper by Kortsh et. all,  
https://royalsocietypublishing.org/doi/10.1098/rspb.2015.1546#d1e354

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import os

In [3]:
# networkx dosen't accept xlsx format, so first import into a pandas dataframe to work with

dataTitle = 'Boreal_and_Arctic_food_webs_of_the_Barents_Sea.xlsx'
excel = pd.ExcelFile(dataTitle)
borealWeb = pd.read_excel(excel, 'The Boreal food web of the BS')
arcticWeb = pd.read_excel(excel, 'The Arctic food web of the BS')

In [4]:
borealLabels = borealWeb.columns.tolist()
arcticLabels = arcticWeb.columns.tolist()

borealNames = list(borealWeb.iloc[:, 0])
arcticNames = list(arcticWeb.iloc[:, 0])

borealSize = borealWeb.shape
arcticSize = arcticWeb.shape

In [5]:
# Dictionaries to make transfering from scientific names to the shortened label

borealNameToLabel = {}
arcticNameToLabel = {}
for i in range(len(borealNames)):
    borealNameToLabel[borealNames[i]] = borealLabels[i + 2]
for i in range(len(arcticNames)):
    arcticNameToLabel[arcticNames[i]] = arcticLabels[i + 2]

In [6]:
# Double checking all species were loaded (Paper cites 233 unique species)

uniqueNames = borealLabels.copy()[2:]
for i in arcticLabels:
    if (i not in uniqueNames):
        uniqueNames.append(i)

print(len(uniqueNames))

235


In [7]:
# If you want to remake the .gml files, set these to True

generateBorealData = True
generateArcticData = True

In [8]:
# Builds the .gml file based on the adjacency matrix given in the .xlsx format

if (generateBorealData):
    with open("Boreal_Data.gml", "w") as gmlFile:
        # initialize the file
        gmlFile.write('graph\n')
        gmlFile.write('[\n')
        gmlFile.write('  directed 1\n')

        # set all nodes to have a numerical id and the abreviation as the label
        for i in range(borealSize[0]):
            gmlFile.write('  node\n  [\n')
            gmlFile.write('    id ' + str(i) + '\n')
            gmlFile.write('    label "' + str(borealWeb.iloc[i, 1]) + '"\n')
            gmlFile.write('  ]\n')

        # set the edges only where there is a 1 in the adjacency matrix
        for i in range(borealSize[0]):
            for j in range(2, borealSize[1]):
                if (borealWeb.iloc[i, j] == 1):
                    gmlFile.write('  edge\n  [\n')
                    gmlFile.write('    source ' + str(i) +'\n')
                    gmlFile.write('    target ' + str(j - 2) + '\n')
                    gmlFile.write('  ]\n')
    
        gmlFile.write(']')

In [9]:
if (generateArcticData):
    with open("Arctic_Data.gml", "w") as gmlFile:
        gmlFile.write('graph\n')
        gmlFile.write('[\n')
        gmlFile.write('  directed 1\n')
    
        for i in range(arcticSize[0]):
            gmlFile.write('  node\n  [\n')
            gmlFile.write('    id ' + str(i) + '\n')
            gmlFile.write('    label "' + str(arcticWeb.iloc[i, 1]) + '"\n')
            gmlFile.write('  ]\n')
    
        for i in range(arcticSize[0]):
            for j in range(2, arcticSize[1]):
                if (arcticWeb.iloc[i, j] == 1):
                    gmlFile.write('  edge\n  [\n')
                    gmlFile.write('    source ' + str(i) +'\n')
                    gmlFile.write('    target ' + str(j - 2) + '\n')
                    gmlFile.write('  ]\n')
    
        gmlFile.write(']')

In [10]:
# load the gml files

borealGraph = nx.read_gml('Boreal_Data.gml', label='id')
arcticGraph = nx.read_gml('Arctic_Data.gml', label='id')
borealGLabels = nx.get_node_attributes(borealGraph, "label")
arcticGLabels = nx.get_node_attributes(arcticGraph, "label")

In [11]:
def fraction_self_feeding(G):
    adj_matrix = nx.to_numpy_array(G)
    self_feeding_count = np.sum(np.diag(adj_matrix) != 0)
    total_species = G.number_of_nodes()
    return self_feeding_count / total_species

In [17]:
#boreal_graph metrics 
S = borealGraph.number_of_nodes()
L = borealGraph.number_of_edges()
C = L/S**2
LD = L/S
meanPath = nx.average_shortest_path_length(borealGraph)
Can = fraction_self_feeding(borealGraph)
# clustering = nx.transitivity(borealGraph)
# modularity = nx.community.greedy_modularity_communities(borealGraph)

print(f'Number of species, S  = {S}')
print(f'Number of links, L  = {L}') 
print(f"Linkage Density: {LD:.2f}")
print(f"Connectance: {C:.2f}")
# print(f"Percentage Omnivores:")
print(f"Percentage Cannibals: {Can:.2f}")
# print(f"Percentage of Species in Loops:")
print(f'Mean shortest path length: %5.2f' % meanPath)
# print (f"Mean Clustering: {clustering}")
# print (f"Modulairty: {modularity}")

Number of species, S  = 180
Number of links, L  = 1546
Linkage Density: 8.59
Connectance: 0.05
Percentage Cannibals: 0.13
Mean shortest path length:  0.63


In [18]:
#arctic_graph metrics 
S = arcticGraph.number_of_nodes()
L = arcticGraph.number_of_edges()
C = L/S**2
LD = L/S
meanPath = nx.average_shortest_path_length(arcticGraph)
Can = fraction_self_feeding(arcticGraph)
# clustering = nx.transitivity(arcticGraph)

print(f'Number of species, S  = {S}')
print(f'Number of links, L  = {L}')
print(f"Linkage Density: {LD:.2f}")
print(f"Connectance: {C:.2f}")
# print(f"Percentage Omnivores:")
print(f"Percentage Cannibals: {Can:.2f}")
# print(f"Percentage of Species in Loops:")
print(f'Mean shortest path length: %5.2f' % meanPath)
# print (f"Mean Clustering: {clustering}")
# print (f"Modulairty:")

Number of species, S  = 159
Number of links, L  = 848
Linkage Density: 5.33
Connectance: 0.03
Percentage Cannibals: 0.06
Mean shortest path length:  0.26


In [12]:
# nx.draw(borealGraph)

In [13]:
# nx.draw(arcticGraph)

In [14]:
# collect the maximum total, in, and out degrees to see which species are the biggest predators or 
# prey in each environment. Below are tables giving the common name of each of these species to better
# understand the data

borealDegreesTotal = {}
borealDegreesIn = {}
borealDegreesOut = {}
for i, j in borealGraph.degree():
    borealDegreesTotal[borealNames[i]] = j
for i, j in borealGraph.in_degree():
    borealDegreesIn[borealNames[i]] = j
for i, j in borealGraph.out_degree():
    borealDegreesOut[borealNames[i]] = j

maximumDegreeBorealTotal = dict(sorted(borealDegreesTotal.items(), key=lambda item: item[1], reverse=True))
maximumDegreeBorealIn = dict(sorted(borealDegreesIn.items(), key=lambda item: item[1], reverse=True))
maximumDegreeBorealOut = dict(sorted(borealDegreesOut.items(), key=lambda item: item[1], reverse=True))
for i in range(10):
    names = list(maximumDegreeBorealTotal.keys())
    print(names[i], ': ', maximumDegreeBorealTotal[names[i]])
print()
for i in range(10):
    names = list(maximumDegreeBorealIn.keys())
    print(names[i], ': ', maximumDegreeBorealIn[names[i]])
print()
for i in range(10):
    names = list(maximumDegreeBorealOut.keys())
    print(names[i], ': ', maximumDegreeBorealOut[names[i]])

Gadus_morhua :  113
Melanogrammus_aeglefinus :  88
Sebastes_mentella :  62
Pandalus_borealis :  59
Clupea_harengus :  58
Paralithodes_camtschaticus :  55
Micromesistius_poutassou :  55
Detritus :  52
Calanus_finmarchicus :  52
Amblyraja_radiata :  50

Gadus_morhua :  82
Melanogrammus_aeglefinus :  65
Paralithodes_camtschaticus :  52
Amblyraja_radiata :  49
Sebastes_mentella :  49
Somniosus_microcephalus :  37
Micromesistius_poutassou :  35
Phocoena_phocoena :  32
Pollachius_virens :  31
Clupea_harengus :  30

Detritus :  52
Phytoplankton_indet :  47
Calanus_finmarchicus :  44
Calanus_hyperboreus :  39
Thysanoessa_inermis :  38
Pandalus_borealis :  36
Calanus_glacialis :  35
Polychaeta :  34
Diatom :  33
Mallotus_villosus :  33


The most interconnected trophospecies in the boreal sea (most total degree):  

|Scientific Name            |Common Name      |Degree |
|---------------------------|-----------------|-------|
|Gadus_morhua               |Atlantic cod     |113    |
|Melanogrammus_aeglefinus   |Haddock          |88     |
|Sebastes_mentella          |Beaked redfish   |62     |
|Pandalus_borealis          |Caridean shrimp  |59     |
|Clupea_harengus            |Atlantic herring |58     |
|Paralithodes_camtschaticus |Red king crab    |55     |
|Micromesistius_poutassou   |Blue whiting     |55     |
|Detritus                   |trash            |52     |
|Calanus_finmarchicus       |zooplankton      |52     |
|Amblyraja_radiata          |Thorny skate     |50     |

The most predatory trophospecies in the boreal sea (most in-degree):

|Scientific Name            |Common Name      |Degree |
|---------------------------|-----------------|-------|
|Gadus_morhua               |Atlantic cod     |82     |
|Melanogrammus_aeglefinus   |Haddock          |65     |
|Paralithodes_camtschaticus |Red king crab    |52     |
|Amblyraja_radiata          |Thorny skate     |49     |
|Sebastes_mentella          |Beaked redfish   |49     |
|Somniosus_microcephalus    |Greenland shark  |37     |
|Micromesistius_poutassou   |Blue whiting     |35     |
|Phocoena_phocoena          |Harbour porpoise |32     |
|Pollachius_virens          |Pollock          |31     |
|Clupea_harengus            |Atlantic herring |30     |

The most preyed upon trophospecies in the boreal sea (most out-degree):

|Scientific Name            |Common Name      |Degree |
|---------------------------|-----------------|-------|
|Detritus                   |trash            |52     |
|Phytoplankton_indet        |phytoplankton    |47     |
|Calanus_finmarchicus       |zooplankton      |44     |
|Calanus_hyperboreus        |copepod          |39     |
|Thysanoessa_inermis        |krill            |38     |
|Pandalus_borealis          |Caridean shrimp  |36     |
|Calanus_glacialis          |copepod          |35     |
|Polychaeta                 |worm             |34     |
|Diatom                     |algae            |33     |
|Mallotus_villosus          |Capelin          |33     |


In [15]:
# How many predators does the freaking shark have
print(borealDegreesOut['Somniosus_microcephalus'])

# what da freak is it
label = borealNameToLabel['Somniosus_microcephalus']

num = borealLabels.index(label) - 2
for i, j in borealGraph.edges:
    if (i == num):
        print(borealNames[i], borealNames[j])

# its canabalism

1
Somniosus_microcephalus Somniosus_microcephalus


In [16]:
arcticDegreesTotal = {}
arcticDegreesIn = {}
arcticDegreesOut = {}
for i, j in arcticGraph.degree():
    arcticDegreesTotal[arcticNames[i]] = j
for i, j in arcticGraph.in_degree():
    arcticDegreesIn[arcticNames[i]] = j
for i, j in arcticGraph.out_degree():
    arcticDegreesOut[arcticNames[i]] = j

maximumDegreeArcticTotal = dict(sorted(arcticDegreesTotal.items(), key=lambda item: item[1], reverse=True))
maximumDegreeArcticIn = dict(sorted(arcticDegreesIn.items(), key=lambda item: item[1], reverse=True))
maximumDegreeArcticOut = dict(sorted(arcticDegreesOut.items(), key=lambda item: item[1], reverse=True))
for i in range(10):
    names = list(maximumDegreeArcticTotal.keys())
    print(names[i], ': ', maximumDegreeArcticTotal[names[i]])
print()
for i in range(10):
    names = list(maximumDegreeArcticIn.keys())
    print(names[i], ': ', maximumDegreeArcticIn[names[i]])
print()
for i in range(10):
    names = list(maximumDegreeArcticOut.keys())
    print(names[i], ': ', maximumDegreeArcticOut[names[i]])

Detritus :  55
Phytoplankton_indet :  44
Boreogadus_saida :  42
Pandalus_borealis :  41
Calanus_finmarchicus :  37
Calanus_hyperboreus :  37
Themisto_libellula :  37
Mallotus_villosus :  35
Thysanoessa_inermis :  34
Polychaeta :  34

Chionoecetes_opilio :  28
Erignathus_barbatus :  27
Boreogadus_saida :  25
Alle_alle :  24
Anarhichas_lupus :  21
Pandalus_borealis :  20
Phoca_hispida :  19
Fulmarus_glacialis :  18
Amblyraja_hyperborea :  17
Hippoglossoides_platessoides :  17

Detritus :  55
Phytoplankton_indet :  44
Calanus_hyperboreus :  33
Polychaeta :  33
Themisto_libellula :  32
Calanus_finmarchicus :  29
Calanus_glacialis :  28
Thysanoessa_inermis :  28
Mallotus_villosus :  24
Diatom :  23


The most interconnected trophospecies in the arctic sea (most total degree):  

|Scientific Name            |Common Name      |Degree |
|---------------------------|-----------------|-------|
|Detritus                   |trash            |55     |
|Phytoplankton_indet        |phytoplankton    |44     |
|Boreogadus_saida           |Arctic cod       |42     |
|Pandalus_borealis          |Caridean shrimp  |41     |
|Calanus_finmarchicus       |zooplankton      |37     |
|Calanus_hyperboreus        |copepod          |37     |
|Themisto_libellula         |amphipod         |37     |
|Mallotus_villosus          |Capelin          |35     |
|Thysanoessa_inermis        |krill            |34     |
|Polychaeta                 |worm             |34     |

The most predatory trophospecies in the arctic sea (most in-degree):

|Scientific Name             |Common Name      |Degree |
|----------------------------|-----------------|-------|
|Chionoecetes_opilio         |Snow crab        |28     |
|Erignathus_barbatus         |Bearded seal     |27     |
|Boreogadus_saida            |Arctic cod       |25     |
|Alle_alle                   |sea bird         |24     |
|Anarhichas_lupus            |Atlantic wolffish|21     |
|Pandalus_borealis           |Caridean shrimp  |20     |
|Phoca_hispida               |Ringed seal      |19     |
|Fulmarus_glacialis          |Northern fulmar  |18     |
|Amblyraja_hyperborea        |Arctic skate     |17     |
|Hippoglossoides_platessoides|American plaice  |17     |

The most preyed upon trophospecies in the arctic sea (most in-degree):

|Scientific Name            |Common Name      |Degree |
|---------------------------|-----------------|-------|
|Detritus                   |trash            |55     |
|Phytoplankton_indet        |phytoplankton    |44     |
|Calanus_hyperboreus        |copepod          |33     |
|Polychaeta                 |worm             |33     |
|Themisto_libellula         |amphipod         |32     |
|Calanus_finmarchicus       |zooplankton      |29     |
|Fulmarus_glacialis         |Northern fulmar  |28     |
|Thysanoessa_inermis        |krill            |28     |
|Mallotus_villosus          |Capelin          |24     |
|Diatom                     |algae            |23     |

## Arctic II

In [22]:
# Find the intersection of the two sets
shared_species = set(borealGLabels.values()) & set(arcticGLabels.values())

# Check if there are any common nodes
if shared_species:
    print(f"The graphs share {len(shared_species)} shared species")
    # print(shared_species)
else:
    print("The graphs do not share any common labels.")

The graphs share 106 shared species


In [20]:
# new fish to be added:
# cod (Gadus morhua), haddock (Melanogrammus aeglefinus), golden redfish (Sebastes norvegicus) and beaked redfish (Sebastes mentella)

new_species = ["GAD_MOR", "MEL_AEG", "SEB_MAR", "SEB_MEN"]

# add new species to account for cannibalism and previous interactions amoungst themselves 
shared_species = set(borealGLabels.values()) & set(arcticGLabels.values())
shared_species.add("GAD_MOR")
shared_species.add("MEL_AEG")
shared_species.add("SEB_MAR")
shared_species.add("SEB_MEN")

arctic_ii_graph = arcticGraph.copy()

for species in new_species:
    arctic_ii_graph.add_node(len(arctic_ii_graph), label=species)

# Function to get node ID by label
def get_node_by_label(graph, label):
    return [node for node, data in graph.nodes(data=True) if data['label'] == label][0]

# Add edges for new fish species based on Boreal graph connections
for fish in new_species:
    fish_node_boreal = get_node_by_label(borealGraph, fish)
    fish_node_arctic = get_node_by_label(arctic_ii_graph, fish)
    for species in shared_species:
        species_node_boreal = get_node_by_label(borealGraph, species)
        species_node_arctic = get_node_by_label(arctic_ii_graph, species)
        if borealGraph.has_edge(fish_node_boreal, species_node_boreal):
            arctic_ii_graph.add_edge(fish_node_arctic, species_node_arctic)
        if borealGraph.has_edge(species_node_boreal, fish_node_boreal):
            arctic_ii_graph.add_edge(species_node_arctic, fish_node_arctic)

print(f"Arctic II graph has {arctic_ii_graph.number_of_nodes()} nodes and {arctic_ii_graph.number_of_edges()} edges")
print(f"New fish species added: {new_species}")

Arctic II graph has 163 nodes and 1014 edges
New fish species added: ['GAD_MOR', 'MEL_AEG', 'SEB_MAR', 'SEB_MEN']


In [23]:
#arctic_II_graph metrics 
S = arctic_ii_graph.number_of_nodes()
L = arctic_ii_graph.number_of_edges()
C = L/S**2
LD = L/S
meanPath = nx.average_shortest_path_length(arctic_ii_graph)
Can = fraction_self_feeding(arctic_ii_graph)
# clustering = nx.transitivity(arcticGraph)

print(f'Number of species, S  = {S}')
print(f'Number of links, L  = {L}')
print(f"Linkage Density: {LD:.2f}")
print(f"Connectance: {C:.2f}")
# print(f"Percentage Omnivores:")
print(f"Percentage Cannibals: {Can:.2f}")
# print(f"Percentage of Species in Loops:")
# print(f'Mean shortest path length: %5.2f' % meanPath)
# print (f"Mean Clustering: {clustering}")
# print (f"Modulairty:")

Number of species, S  = 163
Number of links, L  = 1014
Linkage Density: 6.22
Connectance: 0.04
Percentage Cannibals: 0.08
